In [2]:
#import eland as ed
import requests
import pandas as pd
from pprint import pprint
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
from elasticsearch.client import SqlClient

# Testando API's elasticserach
## Conexão com cluster

In [ ]:
# curl -H "Authorization: ApiKey wrJ3OeMBSf6qhpc1aVHdqQ" https://faaa243eb75a4fc188857a42510fbae6.vectracs.com.br:9243/_cluster/health\?pretty

In [3]:
# Conetando ao cluster elasticsearch
es = Elasticsearch(
    hosts = 'https://faaa243eb75a4fc188857a42510fbae6.vectracs.com.br:9243/',
    api_key = ('vhOG-HoBNHcdUFNsTjHD', 'wrJ3OeMBSf6qhpc1aVHdqQ')
)

In [4]:
# Informações do cluster elasticsearch
es.info()

{'name': 'instance-0000000000',
 'cluster_name': 'faaa243eb75a4fc188857a42510fbae6',
 'cluster_uuid': 'Xyh-HawmTh6RJvDFMQ7RPw',
 'version': {'number': '7.13.4',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'c5f60e894ca0c61cdbae4f5a686d9f08bcefc942',
  'build_date': '2021-07-14T18:33:36.673943207Z',
  'build_snapshot': False,
  'lucene_version': '8.8.2',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [ ]:
# Verificando indices no cluster elasticsearch
for index in sorted(es.indices.get_alias('*').keys()):
    print(index)

In [ ]:
# Pegar dados de um registro
res = es.get(index = 'apm-7.13.4-metric-000001', id = 'mDbtDHsBn1jYROUzuXNM')
pprint(res['_source'])

## Criação de Index e input de Dados

In [16]:
# Criando JSON para input
data = [{"balance": "$2,410.62", "age": 40, "name": "Bettie Buckner", "gender": "female", "company": "RODEOMAD",
         "email": "bettiebuckner@rodeomad.com", "phone": "+1 (857) 491-2461"},
        {"balance": "$1,143.56", "age": 28, "name": "Hanson Gates", "gender": "male", "company": "PEARLESSA",
         "email": "hansongates@pearlessa.com", "phone": "+1 (825) 524-3896"},
        {"balance": "$2,542.95", "age": 20, "name": "Audra Marshall", "gender": "female", "company": "COMTRAIL",
         "email": "audramarshall@comtrail.com", "phone": "+1 (920) 569-2780"},
        {"balance": "$2,235.86", "age": 34, "name": "Milagros Conrad", "gender": "female", "company": "IDEGO",
         "email": "milagrosconrad@idego.com", "phone": "+1 (823) 451-2064"},
        {"balance": "$2,606.95", "age": 34, "name": "Maureen Lopez", "gender": "female", "company": "EVENTEX",
         "email": "maureenlopez@eventex.com", "phone": "+1 (913) 425-3716"}]

# Realizando a criação do index e input dos dados
for a_data in data:
    res = es.index(index = 'my-index', body = a_data)
    print(res)

{'_index': 'my-index', '_type': '_doc', '_id': 'Zv1UCHsBn1jYROUzIXKo', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}
{'_index': 'my-index', '_type': '_doc', '_id': 'Z_1UCHsBn1jYROUzInJ1', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}
{'_index': 'my-index', '_type': '_doc', '_id': 'aP1UCHsBn1jYROUzInLn', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1}
{'_index': 'my-index', '_type': '_doc', '_id': 'af1UCHsBn1jYROUzI3JW', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1}
{'_index': 'my-index', '_type': '_doc', '_id': 'av1UCHsBn1jYROUzI3LI', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 4, '_primary_term': 1}


## Consutas ao elasticsearch
### Usando API elasticsearch

In [6]:
# Usando uma consulta DSL
body = {'query': {'bool': {'must': [{'match': {'gender': 'male'}}, {'range': {'age': {'gte': 25}}}]}}}
res = es.search(index = 'my-index', body = body)
pprint(res)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': 'Z_1UCHsBn1jYROUzInJ1',
                    '_index': 'my-index',
                    '_score': 2.3862944,
                    '_source': {'age': 28,
                                'balance': '$1,143.56',
                                'company': 'PEARLESSA',
                                'email': 'hansongates@pearlessa.com',
                                'gender': 'male',
                                'name': 'Hanson Gates',
                                'phone': '+1 (825) 524-3896'},
                    '_type': '_doc'}],
          'max_score': 2.3862944,
          'total': {'relation': 'eq', 'value': 1}},
 'timed_out': False,
 'took': 0}


In [7]:
# Usando uma consulta Lucene
q = 'gender:female'
res = es.search(index = 'my-index', q = q, size = 2)
pprint(res)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': 'Zv1UCHsBn1jYROUzIXKo',
                    '_index': 'my-index',
                    '_score': 0.2876821,
                    '_source': {'age': 40,
                                'balance': '$2,410.62',
                                'company': 'RODEOMAD',
                                'email': 'bettiebuckner@rodeomad.com',
                                'gender': 'female',
                                'name': 'Bettie Buckner',
                                'phone': '+1 (857) 491-2461'},
                    '_type': '_doc'},
                   {'_id': 'aP1UCHsBn1jYROUzInLn',
                    '_index': 'my-index',
                    '_score': 0.2876821,
                    '_source': {'age': 20,
                                'balance': '$2,542.95',
                                'company': 'COMTRAIL',
                                'email': 'audramarshall@comtrail.com',
 

In [8]:
# Usando sintaxe SQL
query = """ SELECT * FROM "my-index" """
res = SqlClient(es).query(body = {'query': query})   
#res = es.sql.query(body = {'query': query})
res['rows']


[[40,
  '$2,410.62',
  'RODEOMAD',
  'bettiebuckner@rodeomad.com',
  'female',
  'Bettie Buckner',
  '+1 (857) 491-2461'],
 [28,
  '$1,143.56',
  'PEARLESSA',
  'hansongates@pearlessa.com',
  'male',
  'Hanson Gates',
  '+1 (825) 524-3896'],
 [20,
  '$2,542.95',
  'COMTRAIL',
  'audramarshall@comtrail.com',
  'female',
  'Audra Marshall',
  '+1 (920) 569-2780'],
 [34,
  '$2,235.86',
  'IDEGO',
  'milagrosconrad@idego.com',
  'female',
  'Milagros Conrad',
  '+1 (823) 451-2064'],
 [34,
  '$2,606.95',
  'EVENTEX',
  'maureenlopez@eventex.com',
  'female',
  'Maureen Lopez',
  '+1 (913) 425-3716']]

In [19]:
# Transformando consulta em dataframe
columns = ['age', 'balance', 'company', 'email', 'gender', 'name', 'phone']
df_ecommerce2 = pd.DataFrame.from_records(data = res['rows'], columns = columns )
df_ecommerce2

,age,balance,company,email,gender,name,phone
0,40,"$2,410.62",RODEOMAD,bettiebuckner@rodeomad.com,female,Bettie Buckner,+1 (857) 491-2461
1,28,"$1,143.56",PEARLESSA,hansongates@pearlessa.com,male,Hanson Gates,+1 (825) 524-3896
2,20,"$2,542.95",COMTRAIL,audramarshall@comtrail.com,female,Audra Marshall,+1 (920) 569-2780
3,34,"$2,235.86",IDEGO,milagrosconrad@idego.com,female,Milagros Conrad,+1 (823) 451-2064
4,34,"$2,606.95",EVENTEX,maureenlopez@eventex.com,female,Maureen Lopez,+1 (913) 425-3716


In [ ]:
res['columns']
res['columns'][0]
res['columns'][0]['name']
res['columns'][1]['name']

### Usando API elasticsearch_dsl

In [12]:
# Query a ser executada e objeto elasticsearch
query = Q('match', gender = 'male') & Q('range', age = {'gte': 25})
s = Search(using = es, index = 'my-index').query(query)
response = s.execute()

# Response da consulta
for hit in response:
    pprint(hit.to_dict())

{'age': 28,
 'balance': '$1,143.56',
 'company': 'PEARLESSA',
 'email': 'hansongates@pearlessa.com',
 'gender': 'male',
 'name': 'Hanson Gates',
 'phone': '+1 (825) 524-3896'}


In [13]:
# Query a ser executada e objeto elasticsearch
query = Q('match', gender = 'male') & Q('range', age = {'gte': 25})
s = Search(using = es, index = 'my-index').query(query)
response = s.execute()

# Response da consulta
for hit in response:
    print(hit.name)

Hanson Gates


In [14]:
# Transformando consulta em dataframe
df_ecommerce = pd.DataFrame.from_records([hit.to_dict() for hit in s.scan()])
df_ecommerce.head().style.background_gradient(cmap = 'viridis')

,balance,age,name,gender,company,email,phone
0,"$1,143.56",28,Hanson Gates,male,PEARLESSA,hansongates@pearlessa.com,+1 (825) 524-3896


In [15]:
# Buscando todos os registros do index
s = Search(using = es, index = 'my-index').query('match_all')
# response = s.execute()

# Transformando consulta em dataframe
df_ecommerce = pd.DataFrame.from_records([hit.to_dict() for hit in s.scan()])
df_ecommerce.head().style.background_gradient(cmap = 'viridis')

,balance,age,name,gender,company,email,phone
0,"$2,410.62",40,Bettie Buckner,female,RODEOMAD,bettiebuckner@rodeomad.com,+1 (857) 491-2461
1,"$1,143.56",28,Hanson Gates,male,PEARLESSA,hansongates@pearlessa.com,+1 (825) 524-3896
2,"$2,542.95",20,Audra Marshall,female,COMTRAIL,audramarshall@comtrail.com,+1 (920) 569-2780
3,"$2,235.86",34,Milagros Conrad,female,IDEGO,milagrosconrad@idego.com,+1 (823) 451-2064
4,"$2,606.95",34,Maureen Lopez,female,EVENTEX,maureenlopez@eventex.com,+1 (913) 425-3716
